ALL Imports

In [0]:
# Imports

import  pandas as pd
import os
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn import preprocessing

print("Imported All Imports")

Imported All Imports


In [0]:
#os.chdir("godrive/app/predictDamage")

#!ls -l /home/
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
#  !ls -l drive/app/predictDamage
# !cd drive/app/predictDamage
# !pwd

import os
os.chdir("drive/app/predictDamage")
os.listdir()

!ls -l
# !cd drive/app

total 324735
-rw-r--r-- 1 root root  67531978 Jul  6 16:37 Building_Ownership_Use.csv
-rw-r--r-- 1 root root 196802973 Jul  6 16:39 Building_Structure.csv
-rw-r--r-- 1 root root     23657 Jul  7 14:04 PredictDamage.ipynb
-rw-r--r-- 1 root root   8902245 Jul  6 16:39 sample_submission.csv
-rw-r--r-- 1 root root  21684298 Jul  6 16:36 test.csv
-rw-r--r-- 1 root root  37581420 Jul  6 16:36 train.csv


In [0]:
import  pandas as pd
train = pd.read_csv("train.csv")
ownerShip = pd.read_csv("Building_Ownership_Use.csv")
structure = pd.read_csv("Building_Structure.csv")
train_ownership = pd.merge(train, ownerShip, on='building_id', how='inner')
train_ownership_str = pd.merge(train_ownership, structure, on='building_id', how='inner')

# train_ownership_str = pd.read_pickle("train_ownership_str.pkl")
# train_ownership_str.head(2)
# train_ownership_str.shape

In [0]:
X = train_ownership_str[train_ownership_str.columns.difference(['bulding_id','damage_grade'])]
Y = pd.DataFrame(train_ownership_str.loc[:, 'damage_grade'])
colsToExclude = ('building_id')
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(2)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")
    return mis_val_table_ren_columns

missing_values_table(X)

Your selected dataframe has 57 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
has_repair_started,33417,5.29
count_families,1,0.00


In [0]:
def imputeMissingValues(df,categoryMethod,numericalMethod,categoryPackage,numericalPacakge,colsToExclude):
    catModule = __import__(categoryPackage)
    catFunc = getattr(catModule, categoryMethod)
    numModule = __import__(numericalPacakge)
    numFunc = getattr(numModule, numericalMethod)
    whereModule = __import__("numpy")
    whereFunc = getattr(whereModule, "where")
    for col in df:
      if df[col].dtype == 'object' and col not in colsToExclude:
          df[col] = whereFunc(df[col].isnull(),catFunc(df[col]),df[col])
      elif df[col].dtype in ('float64', 'int64', 'int32' ,'float32' ) and col not in colsToExclude:
          df[col] = whereFunc(df[col].isnull(),numFunc(df[col]),df[col])

imputeMissingValues(X,"mode","nanmean","statistics","numpy",colsToExclude)
missing_values_table(X)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


Your selected dataframe has 57 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [0]:
def labelEncodingOfVariables(df, minNoOfUniqueClass, colsToExclude):
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    noOfColsLabeld = 0
    colsLabelled = list()
    for col in df:
        if df[col].dtype == 'object' and col not in colsToExclude:
            if len(list(df[col].unique())) <= minNoOfUniqueClass:
                le.fit(df[col])
                colsLabelled.append(col)
                df[col] = le.transform(df[col])
                noOfColsLabeld += 1
                print("Label Encoding done for %s" % col)
    print("Total Number of coloumns Labelled are %d" % noOfColsLabeld)
    return colsLabelled


colsLabelled_X = labelEncodingOfVariables(X, 3, colsToExclude)
colsLabelled_Y = labelEncodingOfVariables(Y, 5, colsToExclude)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Label Encoding done for land_surface_condition
Label Encoding done for roof_type
Total Number of coloumns Labelled are 2
Label Encoding done for damage_grade
Total Number of coloumns Labelled are 1


In [0]:
def oneHotEncodingOfVariable(data,colsToExclude):
    import pandas as pd
    for col in data:
      if data[col].dtype == 'object' and col not in colsToExclude:
          data = pd.get_dummies(data, columns= [col], prefix=[col], prefix_sep='_')
    return data
X = oneHotEncodingOfVariable(X,('building_id'))


In [0]:
def removeConstantVariables(data,colsToExclude):
    colsRemoved = list()
    for col in data:
      if col not in colsToExclude and data[col].std() == 0:
        colsRemoved.append(col)
        data.drop(col, axis=1, inplace = True)
    return colsRemoved

colsRemoved = removeConstantVariables(X,colsToExclude)


def removeDuplicateVariables(data):
    import numpy as np
    cols = data.columns
    colsScanned = list()
    colsToRemove = list()
    for i in range(len(cols) - 1):
        values = data[cols[i]].values
        dupColoumns = list()
        for j in range(i + 1, len(cols)):
            if np.array_equal(values, data[cols[j]].values):
                colsToRemove.append(cols[j])
                if cols[j] not in colsScanned:
                    dupColoumns.append(cols[j])
                    colsScanned.append(cols[j])
    return colsToRemove


colsToRemove = removeDuplicateVariables(X)
X.drop(colsToRemove,axis=1,inplace=True)


In [0]:
def correlation(dataset, threshold):
    col_corr = {} # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if corr_matrix.iloc[i, j] > threshold:
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr[colname] = corr_matrix.iloc[i, j]
                if colname in dataset.columns:
                   del dataset[colname] # deleting the column from the dataset
    for key, val in col_corr.items():
        print(key, "=>", val)
    # print(dataset)

correlation(X,0.70)


has_geotechnical_risk_landslide => 0.7273914945038104
has_secondary_use_agriculture => 0.7348965503435323
height_ft_post_eq => 0.9410595781983775
height_ft_pre_eq => 0.7735538486810057
vdcmun_id => 0.9996936523100862
ward_id_x => 0.9999999988425701
condition_post_eq_Damaged-Repaired and used => 0.7027756925535288
condition_post_eq_Damaged-Rubble clear => 0.7405041351024928


In [0]:
X.drop('building_id',axis=1,inplace=True)
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.33)


In [0]:
def evaluateClassificationModels(X_train,X_test,Y_train,Y_test):
    #Imports
    from sklearn import linear_model
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    import pandas as pd
    from sklearn import ensemble
    from sklearn import neighbors
    from sklearn.neural_network import MLPClassifier
    from sklearn import tree
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import precision_score,recall_score
    from sklearn.linear_model import SGDClassifier
    from sklearn.linear_model import PassiveAggressiveClassifier
    #Models
    classificationModels = list()
    keysOfParam = set()
#     params_lr = [
#     {'penalty': ['l1'],'C': [1, 10, 100, 1000], 'solver' : ['liblinear','saga']},
#     {'penalty': ['l2'],'C': [1, 10, 100, 1000], 'solver' : ['newton-cg','sag','lbfgs']}
#     ]
#     lr = linear_model.LogisticRegression()
#     classificationModels.append(('Logistic Regression',lr,params_lr))
    params_rf = [
    {'n_estimators' : [10,20,30,50,100],'criterion' : ['gini'],'max_features':['sqrt','log2']},
    {'n_estimators' : [10,20,30,50,100],'criterion' : ['entropy'],'max_features':['sqrt','log2']}
     ]
    rf = ensemble.RandomForestClassifier()
    classificationModels.append(('RandomForest',rf,params_rf))
#     params_kn = [
#        {'n_neighbors' : [1,5,10,15,20,25,30],'weights' : ['uniform'],'algorithm' : ['auto','ball_tree','kd_tree','brute'],'leaf_size' : [5,10,15,20,25,30],'p' : [1,2]},
#        {'n_neighbors' : [1,5,10,15,20,25,30],'weights' : ['distance'],'algorithm' : ['auto','ball_tree','kd_tree','brute'],'leaf_size' : [5,10,15,20,25,30],'p' : [1,2]}
#      ]
#     kn = neighbors.KNeighborsClassifier()
#     classificationModels.append(('KNeighbors',kn,params_kn))
#     params_mlp = [
#        {'hidden_layer_sizes' : [50,75,100,125,150,175,200],'activation' : ['identity','logistic','tanh','relu'],'solver' : ['lbfgs','sgd','adam'],'learning_rate' : ['constant','invscaling','adaptive']},
#        {'hidden_layer_sizes' : [50,75,100,125,150,175,200],'activation' : ['identity','logistic','tanh','relu'],'solver' : ['sgd'],'learning_rate' : ['constant','invscaling','adaptive']}
#      ]
#     mlp = MLPClassifier()
#     classificationModels.append(('MLPClassifier',mlp,params_mlp))

#     params_tr = [
#        {'criterion' : ['gini'],'splitter' : ['best','random'],'max_features' : ['auto','log2']},
#        {'criterion' : ['entropy'] ,'splitter' : ['best','random'],'max_features' : ['auto','log2']}
#      ]
#     tr = tree.DecisionTreeClassifier()
#     classificationModels.append(('DecisionTree',tr,params_tr))

#     params_boost = [
#        {'n_estimators' : [10,20,30,50,100],'random_state' : [5,7,8,9,11]},
#        {'n_estimators' : [60,70,80,90,110],'random_state' : [12,13,14]}
#      ]
#     ada = AdaBoostClassifier()
#     classificationModels.append(('AdaBoostClassifier',ada,params_boost))
#     params_SGD = [
#          {'loss' : ['hinge','log','modified_huber','squared_hinge','perceptron'], 'penalty' : ['l1']},
#          {'loss' : ['hinge','log','modified_huber','squared_hinge','perceptron'], 'penalty' : ['l2']},
#          {'loss' : ['hinge','log','modified_huber','squared_hinge','perceptron'], 'penalty' : ['elasticnet']}
#        ]
#     SGD = SGDClassifier()
#     classificationModels.append(('Stochastic Gradient Classifier',SGD,params_SGD))
#     params_PA = [
#          {'C': [1, 10, 100, 1000],'random_state' : [5,7,8,9,11]}
#        ]
#     PA = PassiveAggressiveClassifier()
#     classificationModels.append(('Passive Aggressive Classifier',PA,params_PA))
    #     params_SVC = [
    #             {'C': [1, 10,],'kernel' : ['rbf'],'degree' : [3]}
    # #          {'C': [1, 10, 100, 1000],'kernel' : ['linear','rbf','poly','sigmoid'],'degree' : [3,4,5,6]}
    #         ]
    #     svc = SVC()
    #     classificationModels.append(('Support Vector Classifier',svc,params_SVC))
    for name,model,params in classificationModels :
        print("------------------Running model -- {}  -------------------".format(name))
        for i in params:
          for j in i.keys():
            keysOfParam.add(j)
        grid = GridSearchCV(model, params,verbose=1, scoring='accuracy', cv=3)
        grid.fit(X_train, Y_train.values.ravel())
        best_parameters = grid.best_estimator_.get_params()
        print("------------------Best Parameters for the model -- {}  -------------------".format(name))
        for param_name in sorted(keysOfParam):
            if param_name in best_parameters.keys():
                print('%s: %r' % (param_name, best_parameters[param_name]))
        predictions = grid.predict(X_test)
        print('####Accuracy:', accuracy_score(Y_test, predictions))
        print('####Precision:', precision_score(Y_test, predictions))
        print('####Recall:', recall_score(Y_test, predictions))
        print("------------------Model came to END -- {}  -------------------".format(name))


In [0]:
evaluateClassificationModels(X_train,X_test,Y_train,Y_test)

------------------Running model -- RandomForest  -------------------
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 42.5min finished


------------------Best Parameters for the model -- RandomForest  -------------------
criterion: 'entropy'
max_features: 'sqrt'
n_estimators: 100
####Accuracy: 0.7241920165769706


ValueError: ignored